In [ ]:

# 📦 라이브러리 설치
# 필요 시: !pip install darts[torch] pandas matplotlib --upgrade

import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import TSMixerModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mae, rmse

# 📊 1. 전력 데이터 불러오기 및 처리
df_power = pd.read_csv("한국전력거래소_시간별 전국 전력수요량_20241231.csv", encoding="cp949")
df_power = df_power.melt(id_vars='날짜', var_name='시간', value_name='수요량(MWh)')
df_power['시간'] = df_power['시간'].str.replace("시", "").astype(int)
df_power['일시'] = pd.to_datetime(df_power['날짜']) + pd.to_timedelta(df_power['시간'] % 24, unit='h')
df_power.loc[df_power['시간'] == 24, '일시'] += pd.Timedelta(days=1)
df_power = df_power[['일시', '수요량(MWh)']].sort_values('일시').set_index('일시')

# 🌦️ 2. 기상 데이터 로딩 함수
def load_weather(filename, colname, start_date="2024-01-01"):
    df = pd.read_csv(filename)
    day_col = [c for c in df.columns if "day" in c][0]
    hour_col = [c for c in df.columns if "hour" in c][0]
    value_col = [c for c in df.columns if "value location" in c][0]
    df = df[pd.to_numeric(df[day_col], errors='coerce').notnull()]
    df[day_col] = df[day_col].astype(int)
    df[hour_col] = df[hour_col].astype(int)
    df['일자'] = pd.to_datetime(start_date) + pd.to_timedelta(df[day_col] - 1, unit='D')
    df['일시'] = df['일자'] + pd.to_timedelta(df[hour_col], unit='h')
    df = df[['일시', value_col]].rename(columns={value_col: colname})
    return df.set_index('일시')

# 3. 모든 기상 데이터 불러오기
df_rain = load_weather("청운효자동_강수_20240101_20241231.csv", "강수량")
df_temp = load_weather("청운효자동_기온_20240101_20241231.csv", "기온")
df_humi = load_weather("청운효자동_습도_20240101_20241231.csv", "습도")
df_wind = load_weather("청운효자동_풍속_20240101_20241231.csv", "풍속")
df_sky  = load_weather("청운효자동_하늘상태_20240101_20241231.csv", "하늘상태")

# 4. 데이터 병합
df_all = df_power.join([df_rain, df_temp, df_humi, df_wind, df_sky], how='inner').dropna()
df_all['분기'] = df_all.index.to_period('Q')

# 🧠 5. TSMixer 모델 학습 및 예측
for 분기, df_quarter in df_all.groupby('분기'):
    print(f"\n==== {분기} TSMixer 예측 ====")
    if len(df_quarter) < 300:
        print("데이터 부족(300 미만)으로 건너뜀")
        continue

    series = TimeSeries.from_dataframe(df_quarter, value_cols='수요량(MWh)')
    covariates = TimeSeries.from_dataframe(df_quarter, value_cols=['강수량', '기온', '습도', '풍속', '하늘상태'])
    train, val = series.split_before(0.8)
    cov_train, cov_val = covariates.split_before(0.8)

    scaler = Scaler()
    train_scaled = scaler.fit_transform(train)
    val_scaled = scaler.transform(val)

    cov_scaler = Scaler()
    cov_train_scaled = cov_scaler.fit_transform(cov_train)
    cov_val_scaled = cov_scaler.transform(cov_val)

    model = TSMixerModel(
        input_chunk_length=24,
        output_chunk_length=12,
        n_epochs=3,
        batch_size=8,
        random_state=42,
        force_reset=True
    )
    model.fit(train_scaled, past_covariates=cov_train_scaled, verbose=False)
    pred = scaler.inverse_transform(model.predict(len(val_scaled), past_covariates=cov_val_scaled))

    print("MAE:", mae(val, pred), "RMSE:", rmse(val, pred))
    plt.figure(figsize=(10,3))
    val.plot(label='실제값')
    pred.plot(label='TSMixer 예측')
    plt.title(f'{분기} TSMixer 예측')
    plt.legend()
    plt.show()
